In [134]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
import pandas as pd
import re
import json
import h2o


from functools import lru_cache

from utils import get_data, Config

from tree import make_tree

In [136]:
def clean_url(url):
    return re.sub(r"browser_id=(\d+)", "browser_id=1", url)

def make_clickable(url):
    # Clickable links for debugging; set browser_id=2 (unknown) to not count them in our real data collection
    url = clean_url(url)
    return f'<a href="{url}" target="_blank">{url}</a>'

In [137]:
@lru_cache(maxsize=None)
def id_to_browser(id):
    return " ".join(df.loc[df["browser_id"] == id].iloc[0][["name", "os", "version", "automation_mode", "headless_mode"]].to_list())

## Load Data + Preprocessing

In [167]:
# Load all data
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Browser".name != 'Unknown';
"""
df = get_data(Config(), initial_data)

Connecting to the PostgreSQL database...
Connection successful


In [168]:
# Extra columns
df["outcome_str"] = df["outcome_value"].fillna("None").astype(str)

# Unify outcomes that are the same
# Firefox: {'error': 'object "TypeError: NetworkError when attempting to fetch resource."', 'headers': ''} Chromium: {'error': 'object "TypeError: Failed to fetch"', 'headers': ''}
# Safari: {'error': 'object "TypeError: Load failed"', 'headers': ''}
df["outcome_str"] = df["outcome_str"].replace("{'error': 'object \"TypeError: NetworkError when attempting to fetch resource.\"', 'headers': ''}", "{'error': 'object \"TypeError: Failed to fetch\"', 'headers': ''}")
df["outcome_str"] = df["outcome_str"].replace("{'error': 'object \"TypeError: Load failed\"', 'headers': ''}", "{'error': 'object \"TypeError: Failed to fetch\"', 'headers': ''}")

# For document referrer we do not want to know the exact resp_id and count
# Maybe we even only want to know whether it is a origin or the full URl?
df['outcome_str'] = df['outcome_str'].replace(r'resp_id=\d+', 'resp_id=<resp_id>', regex=True)
df['outcome_str'] = df['outcome_str'].replace(r'count=\d+', 'count=<count>', regex=True)

df["clean_url"] = df["full_url"].apply(clean_url)

df["browser"] = df["browser_id"].apply(id_to_browser)
df["org_origin"] = df["org_scheme"] + "://" + df["org_host"]
df["resp_origin"] = df["resp_scheme"] + "://" + df["resp_host"]

In [169]:
df["outcome_str"].unique()

array(["{'error': 'null', 'headers': 'access-control-allow-origin,content-length,date,server,test,'}",
       "{'error': 'null', 'headers': 'content-length,'}",
       '{\'error\': \'object "TypeError: Failed to fetch"\', \'headers\': \'\'}',
       "{'error': 'null', 'headers': 'access-control-allow-credentials,content-length,date,server,test,'}",
       "{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-origin,content-length,date,server,test,'}",
       'load', 'message timeout',
       "{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",
       "{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",
       "{'error': 'null', 'headers': 'cache-control,content-type,'}",
      

## Overview

In [170]:
display(df.head(2))
display(df.describe())
display(df.columns)

,id,outcome_type,outcome_value,test_name,test_status,test_message,test_stack,org_scheme,org_host,resp_scheme,...,version,headless_mode,os,automation_mode,add_info,outcome_str,clean_url,browser,org_origin,resp_origin
0,192957,<class 'dict'>,"{'error': 'null', 'headers': 'access-control-a...",fetch_GET,0,None,None,http,sub.headers.websec.saarland,http,...,1.60.114 (119.0.6045.124),headless-new,Ubuntu 22.04,selenium,,"{'error': 'null', 'headers': 'access-control-a...",http://sub.headers.websec.saarland/_hp/tests/f...,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) s...,http://sub.headers.websec.saarland,http://sub.headers.websec.saarland
1,192958,<class 'dict'>,"{'error': 'null', 'headers': 'access-control-a...",fetch_TEST,0,None,None,http,sub.headers.websec.saarland,http,...,1.60.114 (119.0.6045.124),headless-new,Ubuntu 22.04,selenium,,"{'error': 'null', 'headers': 'access-control-a...",http://sub.headers.websec.saarland/_hp/tests/f...,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) s...,http://sub.headers.websec.saarland,http://sub.headers.websec.saarland


,id,test_status,browser_id,testcase_id,response_id,created_at,updated_at,status_code
count,249532.000000,249532.000000,249532.000000,249532.0,249532.000000,249532,249532,249532.000000
mean,322523.384953,0.011189,16.395220,1.0,79.964678,2023-12-20 14:22:35.813785856,2023-12-20 14:22:35.813786880,216.154810
min,192957.000000,0.000000,3.000000,1.0,4.000000,2023-12-20 09:58:40.566554,2023-12-20 09:58:40.566558,200.000000
25%,255339.750000,0.000000,14.000000,1.0,24.000000,2023-12-20 10:13:55.932915712,2023-12-20 10:13:55.932916480,200.000000
50%,318034.500000,0.000000,17.000000,1.0,48.000000,2023-12-20 11:14:35.287669504,2023-12-20 11:14:35.287669504,200.000000
75%,380417.250000,0.000000,20.000000,1.0,153.000000,2023-12-20 11:31:29.765684224,2023-12-20 11:31:29.765684480,200.000000
max,481836.000000,2.000000,23.000000,1.0,218.000000,2023-12-21 11:01:22.079483,2023-12-21 11:01:22.079484,302.000000
std,79429.848218,0.149174,4.821591,0.0,66.753459,NaN,NaN,37.240009


Index(['id', 'outcome_type', 'outcome_value', 'test_name', 'test_status',
       'test_message', 'test_stack', 'org_scheme', 'org_host', 'resp_scheme',
       'resp_host', 'relation_info', 'browser_id', 'testcase_id',
       'response_id', 'status', 'full_url', 'created_at', 'updated_at',
       'raw_header', 'status_code', 'label', 'resp_type', 'name', 'version',
       'headless_mode', 'os', 'automation_mode', 'add_info', 'outcome_str',
       'clean_url', 'browser', 'org_origin', 'resp_origin'],
      dtype='object')

In [171]:
# Collected results per browser
df.groupby(["browser_id", "name", "version", "os", "headless_mode", "automation_mode"])["id"].count().sort_values(ascending=False).to_frame()

,,,,,,id
browser_id,name,version,os,headless_mode,automation_mode,
15,edge,119,Ubuntu 22.04,headless-new,selenium,20912
14,firefox,119,Ubuntu 22.04,headless,selenium,20912
13,chrome,119,Ubuntu 22.04,headless-new,selenium,20912
19,firefox,119,Ubuntu 22.04,xvfb,selenium,20912
18,chrome,119,Ubuntu 22.04,xvfb,selenium,20912
17,brave,1.60.114 (119.0.6045.124),Ubuntu 22.04,headless-new,selenium,20912
16,brave,1.59.120 (118.0.5993.88),Ubuntu 22.04,headless-new,selenium,20912
20,edge,119,Ubuntu 22.04,xvfb,selenium,20912
21,brave,1.59.120 (118.0.5993.88),Ubuntu 22.04,xvfb,selenium,20912


In [174]:
# Timeouts
display(df.groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"])["test_status"].value_counts().to_frame())

display(df.loc[df["test_status"] == 2].head(3))

count
automation_mode browser_id name         version                   os           headless_mode test_status       
intent          3          brave        1.60.116                  Android 11   real          0             8745
                                                                                             2             1266
                4          ucmobile     13.3.8.1305               Android 11   real          0              206
                                                                                             2               47
                5          opera        78.4.4143.75735           Android 11   real          0            10456
                12         chrome       120.0.6099.44             Android 11   real          0             9519
                23         firefox_beta 122.0b1                   Android 11   real          0            10108
                                                                                             2               65
selenium        13         chrome       119                       Ubuntu 22.04 headless-new  0            20912
                14         firefox      119                       Ubuntu 22.04 headless      0            20911
                                                                                             2                1
                15         edge         119                       Ubuntu 22.04 headless-new  0            20910
                                                                                             2                2
                16         brave        1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  0            20912
                17         brave        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  0            20912
                18         chrome       119                       Ubuntu 22.04 xvfb          0            20912
                19         firefox      119                       Ubuntu 22.04 xvfb          0            20907
                                                                                             2                5
                20         edge         119                       Ubuntu 22.04 xvfb          0            20902
                                                                                             2               10
                21         brave        1.59.120 (118.0.5993.88)  Ubuntu 22.04 xvfb          0            20912
                22         brave        1.60.114 (119.0.6045.124) Ubuntu 22.04 xvfb          0            20912

,id,outcome_type,outcome_value,test_name,test_status,test_message,test_stack,org_scheme,org_host,resp_scheme,...,version,headless_mode,os,automation_mode,add_info,outcome_str,clean_url,browser,org_origin,resp_origin
34765,227596,<class 'NoneType'>,None,oac_window.open,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,119,headless-new,Ubuntu 22.04,selenium,,None,http://sub.headers.websec.saarland/_hp/tests/o...,edge Ubuntu 22.04 119 selenium headless-new,http://sub.headers.websec.saarland,https://headers.webappsec.eu
37784,230593,<class 'NoneType'>,None,oac_window.open,2,Test timed out,None,http,sub.headers.websec.saarland,http,...,119,xvfb,Ubuntu 22.04,selenium,,None,http://sub.headers.websec.saarland/_hp/tests/o...,edge Ubuntu 22.04 119 selenium xvfb,http://sub.headers.websec.saarland,http://headers.webappsec.eu
37785,230594,<class 'NoneType'>,None,oac_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,119,xvfb,Ubuntu 22.04,selenium,,None,http://sub.headers.websec.saarland/_hp/tests/o...,edge Ubuntu 22.04 119 selenium xvfb,http://sub.headers.websec.saarland,https://headers.webappsec.eu


In [175]:
# Time taken!
# NOTE: this is now broken as we ran all tests on linux twice for a stability analysis
df.groupby(["name", "version", "os", "headless_mode", "automation_mode", "org_scheme"])["created_at"].agg(["min", "max"]).apply(lambda x: x["max"] - x["min"], axis=1).to_frame()

0
name         version                   os           headless_mode automation_mode org_scheme                       
brave        1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium        http       0 days 01:29:15.190542
                                                                                  https      0 days 01:26:48.371864
                                                    xvfb          selenium        http       0 days 01:30:28.246903
                                                                                  https      0 days 01:28:25.877146
             1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium        http       0 days 01:29:15.177088
                                                                                  https      0 days 01:26:53.931604
                                                    xvfb          selenium        http       0 days 01:30:25.268345
                                                                                  https      0 days 01:28:07.876485
             1.60.116                  Android 11   real          intent          http       0 days 00:36:59.177528
                                                                                  https      0 days 00:36:59.748971
chrome       119                       Ubuntu 22.04 headless-new  selenium        http       0 days 01:29:06.393217
                                                                                  https      0 days 01:26:43.887262
                                                    xvfb          selenium        http       0 days 01:30:11.077591
                                                                                  https      0 days 01:28:03.952745
             120.0.6099.44             Android 11   real          intent          http       0 days 00:08:31.169707
                                                                                  https      0 days 00:08:29.109817
edge         119                       Ubuntu 22.04 headless-new  selenium        http       0 days 01:23:53.035976
                                                                                  https      0 days 01:27:01.534731
                                                    xvfb          selenium        http       0 days 01:24:54.645264
                                                                                  https      0 days 01:28:17.297944
firefox      119                       Ubuntu 22.04 headless      selenium        http       0 days 01:25:19.112014
                                                                                  https      0 days 01:28:16.290569
                                                    xvfb          selenium        http       0 days 01:25:46.942611
                                                                                  https      0 days 01:27:57.987121
firefox_beta 122.0b1                   Android 11   real          intent          http       0 days 00:08:27.297392
                                                                                  https      0 days 00:08:22.437948
opera        78.4.4143.75735           Android 11   real          intent          http       0 days 00:08:22.086614
                                                                                  https      0 days 00:08:21.245346
ucmobile     13.3.8.1305               Android 11   real          intent          http       0 days 00:14:15.185412
                                                                                  https      0 days 00:11:42.139771

## Response analysis

- We have 10456 unique tests: (test_name x relation_info) x org_scheme x (resp_scheme x resp_host) x response_id)

In [176]:
# Unique tests
tests = {
    "Fetch": [4, 2, 8, 6], 
    "Framing": [9, 2, 8, 35], # 7+11+17
    "PP": [4, 2, 8, 24],
    "OAC": [3, 2, 8, 14],
    "TAO": [1, 2, 8, 13],
    "RP": [2, 2, 8, 16],
    "CSP-SCRIPT": [2, 2, 8, 17],
    "COEP": [2, 2, 8, 11],
    "CORP": [4, 2, 8, 11],
    "CSP-IMG": [1, 2, 8, 17],
    "HSTS": [2, 1, 2, 14],
    "COOP": [1, 2, 8, 11],
}
tests = pd.DataFrame.from_dict(tests, columns=["tests", "org_origins", "resp_origins", "responses"], orient="index")
tests["All tests"] = tests.prod(axis=1)
tests.loc["Sum"] = tests.sum(axis=0)
tests

,tests,org_origins,resp_origins,responses,All tests
Fetch,4,2,8,6,384
Framing,9,2,8,35,5040
PP,4,2,8,24,1536
OAC,3,2,8,14,672
TAO,1,2,8,13,208
RP,2,2,8,16,512
CSP-SCRIPT,2,2,8,17,544
COEP,2,2,8,11,352
CORP,4,2,8,11,704
CSP-IMG,1,2,8,17,272


In [177]:
# Only consider the tests that did not timeout! (we already reran them above)
df = df.loc[df["test_status"] == 0]

In [178]:
# SELECT count(*) as c2, count(DISTINCT browser_id), count(DISTINCT outcome_value) as c1, test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id 
# FROM "Result" GROUP by test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id order by c1 DESC, c2 DESC

res = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])[["browser_id", "outcome_str"]].agg(["count", "nunique"])
display(res)
display(res.value_counts().to_frame())

browser_id  \
                                                                                                                                    count   
test_name             org_scheme org_host                    resp_scheme resp_host                   relation_info response_id              
accesswindow_direct   http       sub.headers.websec.saarland http        headers.webappsec.eu        direct        71                  23   
                                                                                                                   72                  24   
                                                                                                                   73                  23   
                                                                                                                   74                  24   
                                                                                                                   75                  23   
...                                                                                                                                   ...   
upgradeHSTS_subdomain http       sub.headers.websec.saarland https       sub.headers.websec.saarland subdomain     139                 24   
                                                                                                                   140                 24   
                                                                                                                   141                 24   
                                                                                                                   142                 24   
                                                                                                                   143                 24   

                                                                                                                                        \
                                                                                                                               nunique   
test_name             org_scheme org_host                    resp_scheme resp_host                   relation_info response_id           
accesswindow_direct   http       sub.headers.websec.saarland http        headers.webappsec.eu        direct        71               13   
                                                                                                                   72               14   
                                                                                                                   73               13   
                                                                                                                   74               14   
                                                                                                                   75               13   
...                                                                                                                                ...   
upgradeHSTS_subdomain http       sub.headers.websec.saarland https       sub.headers.websec.saarland subdomain     139              14   
                                                                                                                   140              14   
                                                                                                                   141              14   
                                                                                                                   142              14   
                                                                                                                   143              14   

                                                                                                                               outcome_str  \
                                                                             

count
(browser_id, count) (browser_id, nunique) (outcome_str, count) (outcome_str, nunique)       
24                  14                    24                   1                        5733
23                  13                    23                   1                        2836
24                  14                    24                   2                         623
22                  12                    22                   1                         338
23                  13                    23                   2                         219
24                  14                    24                   3                         141
                                                               4                         121
25                  14                    25                   1                         106
23                  13                    23                   3                          67
27                  14                    27                   1                          44
28                  14                    28                   1                          28
30                  14                    30                   2                          27
                                                               3                          21
25                  15                    25                   3                          18
                                                               4                          15
21                  11                    21                   1                          14
26                  15                    26                   3                          13
30                  14                    30                   1                          12
25                  14                    25                   2                          12
22                  13                    22                   1                          10
27                  14                    27                   3                           9
                                                               2                           7
30                  14                    30                   4                           6
38                  14                    38                   1                           6
26                  15                    26                   4                           4
28                  14                    28                   3                           4
23                  14                    23                   1                           4
31                  14                    31                   1                           3
28                  14                    28                   2                           3
26                  13                    26                   2                           3
                                                               1                           2
37                  13                    37                   1                           2
21                  13                    21                   1                           1
                    12                    21                   1                           1
26                  13                    26                   3                           1
30                  13                    30                   1                           1
36                  12                    36                   1                           1

## Stability analysis
- Does the same browser/env always result in the same outcomes?
- We ran all tests on linux twice, does any of these tests have more than one result?
    - A total of 20/10456 tests have different outcomes
    - ~~2xreferrer_iframe (both timeout of the test "None" vs "message timeout" result); can be fixed by removing timeouts first~~
    - 18x subresourceloadingCOEP_img, all differences: 'swag-same-origin.jpg': 'error' | 'load'
        - trying to manually reproduce: e.g., http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=2&label=COEP&first_id=57&last_id=57&scheme=http
        - devtools open: with cache disabeled: 15x error, without cache disabled: 13x or 16x error?
        - devtools closed: 13x or 16x error (tried to fix with Cache-Control: no-cache, no-store but does not seem to have an effect)
        - chrome always 14x error
    - 13x firefox xvfb, 6x firefox headless, 1x edge

In [179]:
stab = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser_id"])[["outcome_str"]].agg(["count", "nunique"])
# Tests with more than one outcome
diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
print(f"{len(diff_outcomes)} tests have different outcomes!")
display(diff_outcomes.reset_index()["browser_id"].apply(id_to_browser).value_counts().to_frame())
display(diff_outcomes)

30 tests have different outcomes!


,count
browser_id,
firefox Ubuntu 22.04 119 selenium headless,18
firefox Ubuntu 22.04 119 selenium xvfb,12


outcome_str  \
                                                                                                                                                         count   
test_name                  org_scheme org_host                    resp_scheme resp_host                       relation_info response_id browser_id               
subresourceloadingCOEP_img http       sub.headers.websec.saarland http        headers.websec.saarland         direct        58          19                   2   
                                                                                                                            60          19                   2   
                                                                                                                            61          14                   2   
                                                                                                                                        19                   2   
                                                                                                                            62          14                   2   
                                                                                                                            63          14                   2   
                                                                                                                            65          14                   2   
                                                                                                                                        19                   2   
                                                                                                                            66          14                   2   
                                                                              sub.headers.websec.saarland     direct        58          19                   2   
                                                                                                                            60          19                   2   
                                                                                                                            61          14                   2   
                                                                                                                                        19                   2   
                                                                                                                            62          14                   2   
                                                                                                                            63          14                   2   
                                                                                                                            65          14                   2   
                                                                                                                                        19                   2   
                                                                                                                            66          14                   2   
                                                                              sub.sub.headers.websec.saarland direct        58          19                   2   
                                                                                                                            60          19                   2   
                                                                                                                            61          14                   2   
                                                                                                                                        19                   2   
                                                                                                 

In [180]:
for test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id, browser_id in stab.loc[stab[("outcome_str", "nunique")] != 1].index:
    print(test_name)
    rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"] == response_id) & (df["browser_id"] == browser_id)]
    with pd.option_context("display.max_colwidth", 200):
        #display(rows[["outcome_str", "name", "headless_mode", "os", "version"]].value_counts().to_frame())
        display(rows[["outcome_str", "name"]].value_counts().to_frame())
        disp = rows.drop_duplicates(subset="outcome_str")[["browser_id", "headless_mode", "name", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
        disp = disp.style.format({'full_url': make_clickable})
        display(disp)
        

subresourceloadingCOEP_img


,,count
outcome_str,name,
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",firefox,1
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",firefox,1


,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
43675,19,xvfb,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,http,sub.headers.websec.saarland,http,direct,headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=1&label=COEP&first_id=58&last_id=58&scheme=http
187829,19,xvfb,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,http,sub.headers.websec.saarland,http,direct,headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?timeout=5&resp_type=basic&browser_id=1&label=COEP&first_id=58&last_id=58&scheme=http


subresourceloadingCOEP_img


,,count
outcome_str,name,
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",firefox,1
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",firefox,1


...

,,count
outcome_str,name,
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",firefox,1
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",firefox,1


,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
91899,14,headless,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,https,sub.headers.websec.saarland,https,direct,sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=1&label=COEP&first_id=58&last_id=58&scheme=https
237550,14,headless,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,https,sub.headers.websec.saarland,https,direct,sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?timeout=5&resp_type=basic&browser_id=1&label=COEP&first_id=58&last_id=58&scheme=https


subresourceloadingCOEP_img


,,count
outcome_str,name,
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",firefox,1
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",firefox,1


,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
91907,14,headless,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,https,sub.headers.websec.saarland,https,direct,sub.sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=1&label=COEP&first_id=58&last_id=58&scheme=https
237558,14,headless,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,https,sub.headers.websec.saarland,https,direct,sub.sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?timeout=5&resp_type=basic&browser_id=1&label=COEP&first_id=58&last_id=58&scheme=https


## Differences:
- COOP (accesswindow_direct):
    - [ ] Edge + Firefox: redirect first
    - [ ] Brave + Chrome: COOP first?
-  Framing:
    -  [ ] framing_iframe_sandbox: CSP-FA *; Firefox allows, all other browsers deny?
- PP not supported in Firefox
- OAC not supported in Firefox
    - [ ] Also random/caching behavior?
- PerfAPI:
    - [ ] Chrome/Firefox have different rules when to create an entry at all?
- COEP (CORP):
    - [ ] Firefox incorrect behavior for same-origin (applies same-site?)
- CORP:
    - [ ] same-origin: Edge does not check the scheme?
    - [x] Firefox applies CORP on object tags (even though it should not!)
    - [ ] Some other CORP and/or COEP strange stuff (Firefox)
- Referrer Policy: different default in Brave (unsafe-url is never allowed)
- Subresource-loading (CSP): differences due to HTTPS Upgrades
- Script Execution (CSP):
    - [ ] Safari sandbox bug?! (maybe related https://bugs.webkit.org/show_bug.cgi?id=223848 unsure though)
- Upgrade (HSTS):
    - [ ] Safari bug?

In [181]:
def get_uniques(df):
    browsers = f"{df['name'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df.groupby(["test_name", "relation_info"]):
    
    unique_outcomes = {}
    group.groupby(["name"])["outcome_str"].value_counts(normalize=True).reset_index().groupby(["outcome_str", "proportion"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        print(grouping)
        # display(unique_outcomes)
        display(group.groupby(["name"])["outcome_str"].value_counts(normalize=True).to_frame().unstack(0))
        for value, dic in unique_outcomes.items():
            values = [f"{v} ({k})" for k,v in dic.items()]
            # print(f"{value}: {values}")
        display(d)
        print()


('accesswindow_direct', 'direct')


proportion            \
name                                                    brave    chrome   
outcome_str                                                               
{'window.open.opener': 'null'}                       0.310881  0.340909   
{'window.open.opener': 'object "TypeError: Cann...   0.088083       NaN   
{'window.open.opener': 'object "TypeError: Cann...        NaN       NaN   
{'window.open.opener': 'object "[object Window]"'}   0.601036  0.659091   

                                                                        \
name                                                    edge   firefox   
outcome_str                                                              
{'window.open.opener': 'null'}                      0.181818  0.181818   
{'window.open.opener': 'object "TypeError: Cann...       NaN       NaN   
{'window.open.opener': 'object "TypeError: Cann...       NaN       NaN   
{'window.open.opener': 'object "[object Window]"'}  0.818182  0.818182   

                                                                           \
name                                               firefox_beta     opera   
outcome_str                                                                 
{'window.open.opener': 'null'}                         0.183432  0.181818   
{'window.open.opener': 'object "TypeError: Cann...          NaN       NaN   
{'window.open.opener': 'object "TypeError: Cann...          NaN       NaN   
{'window.open.opener': 'object "[object Window]"'}     0.816568  0.818182   

                                                             
name                                               ucmobile  
outcome_str                                                  
{'window.open.opener': 'null'}                          NaN  
{'window.open.opener': 'object "TypeError: Cann...      NaN  
{'window.open.opener': 'object "TypeError: Cann...      1.0  
{'window.open.opener': 'object "[object Window]"'}      NaN

,"['edge', 'firefox', 'opera']",['firefox_beta'],['brave'],['chrome'],['ucmobile']
{'window.open.opener': 'null'},0.181818,0.183432,0.310881,0.340909,NaN
"{'window.open.opener': 'object ""[object Window]""'}",0.818182,0.816568,0.601036,0.659091,NaN
"{'window.open.opener': 'object ""TypeError: Cannot read properties of null (reading \'opener\')""'}",NaN,NaN,0.088083,NaN,NaN
"{'window.open.opener': 'object ""TypeError: Cannot read property \'opener\' of null""'}",NaN,NaN,NaN,NaN,1.0



('fetch_GET', 'credentials')


proportion            \
name                                                    brave    chrome   
outcome_str                                                               
{'error': 'null', 'headers': 'access-control-al...   0.020047  0.020833   
{'error': 'null', 'headers': 'access-control-al...   0.020047  0.020833   
{'error': 'null', 'headers': 'access-control-al...   0.020047  0.020833   
{'error': 'null', 'headers': 'access-control-al...   0.022406  0.023148   
{'error': 'null', 'headers': 'access-control-al...   0.021226  0.020833   
{'error': 'null', 'headers': 'content-length,te...   0.028302  0.034722   
{'error': 'object "TypeError: Failed to fetch"'...   0.867925  0.858796   

                                                                        \
name                                                    edge   firefox   
outcome_str                                                              
{'error': 'null', 'headers': 'access-control-al...  0.020833  0.020833   
{'error': 'null', 'headers': 'access-control-al...  0.020833  0.020833   
{'error': 'null', 'headers': 'access-control-al...  0.020833  0.020833   
{'error': 'null', 'headers': 'access-control-al...  0.020833  0.020833   
{'error': 'null', 'headers': 'access-control-al...  0.020833  0.020833   
{'error': 'null', 'headers': 'content-length,te...  0.031250  0.031250   
{'error': 'object "TypeError: Failed to fetch"'...  0.864583  0.864583   

                                                                           \
name                                               firefox_beta     opera   
outcome_str                                                                 
{'error': 'null', 'headers': 'access-control-al...     0.020833  0.020833   
{'error': 'null', 'headers': 'access-control-al...     0.020833  0.020833   
{'error': 'null', 'headers': 'access-control-al...     0.020833  0.020833   
{'error': 'null', 'headers': 'access-control-al...     0.020833  0.020833   
{'error': 'null', 'headers': 'access-control-al...     0.020833  0.020833   
{'error': 'null', 'headers': 'content-length,te...     0.031250  0.031250   
{'error': 'object "TypeError: Failed to fetch"'...     0.864583  0.864583   

                                                             
name                                               ucmobile  
outcome_str                                                  
{'error': 'null', 'headers': 'access-control-al...      NaN  
{'error': 'null', 'headers': 'access-control-al...  0.03125  
{'error': 'null', 'headers': 'access-control-al...  0.03125  
{'error': 'null', 'headers': 'access-control-al...      NaN  
{'error': 'null', 'headers': 'access-control-al...  0.06250  
{'error': 'null', 'headers': 'content-length,te...      NaN  
{'error': 'object "TypeError: Failed to fetch"'...  0.87500

...

proportion                                \
name                                brave    chrome      edge   firefox   
outcome_str                                                               
{'response.redirected': False}       0.82  0.821429  0.821429  0.821429   
{'response.redirected': True}        0.18  0.178571  0.178571  0.178571   

                                                       
name                           firefox_beta     opera  
outcome_str                                            
{'response.redirected': False}     0.821429  0.821429  
{'response.redirected': True}      0.178571  0.178571

,['brave'],"['chrome', 'edge', 'firefox', 'firefox_beta', 'opera']"
{'response.redirected': False},0.82,0.821429
{'response.redirected': True},0.18,0.178571



('upgradeHSTS_subdomain', 'subdomain')


proportion                                \
name                                brave    chrome      edge   firefox   
outcome_str                                                               
{'response.redirected': False}      0.892  0.892857  0.892857  0.892857   
{'response.redirected': True}       0.108  0.107143  0.107143  0.107143   

                                                       
name                           firefox_beta     opera  
outcome_str                                            
{'response.redirected': False}     0.892857  0.892857  
{'response.redirected': True}      0.107143  0.107143

,['brave'],"['chrome', 'edge', 'firefox', 'firefox_beta', 'opera']"
{'response.redirected': False},0.892,0.892857
{'response.redirected': True},0.108,0.107143


In [182]:
test_name_seen = set()
for test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id in res.loc[res[("outcome_str", "nunique")] != 1].index:
    # Only look at each feature group once
    if test_name in test_name_seen:
        continue
    test_name_seen.add(test_name)
    print(test_name)
    rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"] == response_id)]
    with pd.option_context("display.max_colwidth", 200):
        #display(rows[["outcome_str", "name", "headless_mode", "os", "version"]].value_counts().to_frame())
        display(rows[["outcome_str", "name"]].value_counts().to_frame())
        disp = rows.drop_duplicates(subset="outcome_str")[["browser_id", "headless_mode", "name", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
        disp = disp.style.format({'full_url': make_clickable})
        display(disp)
        
    

accesswindow_direct


count
outcome_str                                                                                       name               
{'window.open.opener': 'null'}                                                                    brave             8
                                                                                                  chrome            5
{'window.open.opener': 'object "[object Window]"'}                                                edge              4
                                                                                                  firefox           4
{'window.open.opener': 'object "TypeError: Cannot read properties of null (reading \'opener\')"'} brave             1
{'window.open.opener': 'object "[object Window]"'}                                                firefox_beta      1
                                                                                                  opera             1

,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
44473,15,headless-new,edge,"{'window.open.opener': 'object ""[object Window]""'}","[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups']]",accesswindow_direct,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?resp_type=basic&browser_id=1&label=COOP&first_id=72&last_id=72&scheme=http
57907,13,headless-new,chrome,{'window.open.opener': 'null'},"[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups']]",accesswindow_direct,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?resp_type=basic&browser_id=1&label=COOP&first_id=72&last_id=72&scheme=http
134593,3,real,brave,"{'window.open.opener': 'object ""TypeError: Cannot read properties of null (reading \'opener\')""'}","[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups']]",accesswindow_direct,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?timeout=125&resp_type=basic&browser_id=1&label=COOP&first_id=72&last_id=72&scheme=http&first_popup=7&last_popup=7&run_no_popup=no


framing_iframe


count
outcome_str     name               
message timeout brave             9
                chrome            5
                edge              4
message send    firefox           4
                firefox_beta      1
message timeout opera             1

...

count
outcome_str name               
load        brave             9
            chrome            5
            edge              4
error       firefox           4
            firefox_beta      1
load        opera             1

,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
89130,15,headless-new,edge,load,"[['Cross-Origin-Resource-Policy', 'same-origin']]",subresourceloadingCORP_img,https,sub.headers.websec.saarland,http,direct,sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?resp_type=basic&browser_id=1&label=CORP&first_id=46&last_id=46&scheme=https
94460,14,headless,firefox,error,"[['Cross-Origin-Resource-Policy', 'same-origin']]",subresourceloadingCORP_img,https,sub.headers.websec.saarland,http,direct,sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?resp_type=basic&browser_id=1&label=CORP&first_id=46&last_id=46&scheme=https


subresourceloadingCORP_object


count
outcome_str name               
load        brave             9
            chrome            5
            edge              4
error       firefox           4
            firefox_beta      1
load        opera             1

,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
41716,15,headless-new,edge,load,"[['Cross-Origin-Resource-Policy', 'same-origin']]",subresourceloadingCORP_object,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?resp_type=basic&browser_id=1&label=CORP&first_id=46&last_id=46&scheme=http
43727,14,headless,firefox,error,"[['Cross-Origin-Resource-Policy', 'same-origin']]",subresourceloadingCORP_object,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?resp_type=basic&browser_id=1&label=CORP&first_id=46&last_id=46&scheme=http


## Clustering and co.
- Maybe try decision trees again?
- One tree for each feature group?/label?/test_name?
    - outcome = outcome_str (mostly binary, sometimes more options)
    - variables:
        - browser (includes os, automation mode, headless mode, version)
        - test_id = test_name + relation_info (or only relation_info if one tree per test_name is created)
        - org_orgin = org_scheme + org_host
        - resp_origin = resp_scheme + resp_host
        - response_id or raw_headers + status_code
        - ...
     
- H2O setup:
    - `sudo apt install openjdk-17-jre-headless graphviz`
    - `wget https://h2o-release.s3.amazonaws.com/h2o/rel-3.44.0/2/h2o-3.44.0.2.zip`
    - `unzip h2o-3.44.0.2.zip`
    - `cd h2o-3.44.0.2` and `java -jar h2o.jar`

In [183]:
df[["test_name", "label", "relation_info"]].value_counts()

test_name                      label       relation_info 
fullscreen_iframe              PP          child_allow       9104
                                           child             9104
                                           direct            9104
                                           child_sandbox     9104
imgloading_iframe              CSP-IMG     direct            6464
script_execution_iframe        CSP-SCRIPT  direct            6464
                                           sandbox           6464
framing_iframe                 CSP-FA      sandbox           6456
                                           nested            6456
                                           direct            6456
framing_embed                  CSP-FA      sandbox           6456
framing_object                 CSP-FA      direct            6456
                                           sandbox           6456
                                           nested            6456
framing_embed     

In [185]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()


# Only keep one row for each test that had non-deterministic results (this should only be Firefox COEP same-origin 18x)
print(len(df))
tree_df = df.drop_duplicates(subset=["test_name", "relation_info", "browser", "org_origin", "resp_origin", "response_id"])
print(len(tree_df))

# Idea limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = tree_df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])["outcome_str"].transform("nunique") != 1
tree_df = tree_df.loc[condition]
print(len(tree_df))

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    # pred_cols = ["browser", "relation_info", "org_origin", "resp_origin", "status_code", "raw_header"]
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]

    group = group[["outcome_str", *pred_cols]]
    # Convert all columns to str, then to category
    tree = make_tree(group, pred_cols, group_name)
    # print(tree)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,33 mins 13 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_ubuntu_uy37br
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


248136
142570
18128
('accesswindow_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,33 mins 14 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_ubuntu_uy37br
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('accesswindow_direct', 'direct'), datapoints: 2017
('framing_iframe', 'sandbox')
Connecting to H2O server at http://localhost:54321 ... successful.


...

Create tree: ('subresourceloadingCOEP_img', 'direct'), datapoints: 1184
('subresourceloadingCORP_img', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,33 mins 28 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_ubuntu_uy37br
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('subresourceloadingCORP_img', 'direct'), datapoints: 14
('subresourceloadingCORP_object', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,33 mins 29 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_ubuntu_uy37br
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('subresourceloadingCORP_object', 'direct'), datapoints: 439


# Crash analysis OLD

### Exceptions thrown
- Fixed:
    - Only Firefox; both headless + Xvfb
        - OAC + RP
        - Both on HTTP and HTTPS
        - Response_ids: 185, 186, 341, 342; these are the ones that have status code 302!
        - Reason seems to be that Firefox switches to another tab in these cases and thus the wait for crashes. Solution: forcefully switch back to the original tab before waiting!
    - Only Brave: both headless + Xvfb
        - OAC + RP
        - Both on HTTP and HTTPS
        - Reason seems to be the tab switching, if deactivated no issue occurs

In [165]:
with open("/home/ubuntu/wpt/_hp/tools/crawler/desktop-selenium-2023-11-20 16:09:24.649008.log", "r") as f:
    crashes = []
    for l in f.readlines():
        if "649008: http" in l:
            match = re.match(".*(http|https)://.*browser_id=(\d+)&label=(.*)&first_id=(\d+).*", l)
            crashes.append([match[1], match[2], match[3], match[4]])

In [166]:
crashes = pd.DataFrame(crashes, columns=["scheme", "browser_id", "label", "response_id"]).drop_duplicates()
crashes

,scheme,browser_id,label,response_id
0,http,59,RP,172
2,http,59,RP,173
4,https,59,OAC,339
6,https,59,RP,181
8,http,75,RP,171
10,http,75,RP,172
12,https,75,OAC,330
14,https,75,OAC,338
16,https,75,RP,180
18,https,74,OAC,337
